In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch08Joins").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("ch08Joins").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_16447


In [5]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")

/home/kranthidr/.local/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.7.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)


In [6]:
person.show(5, False)

+---+----------------+----------------+---------------+
|id |name            |graduate_program|spark_status   |
+---+----------------+----------------+---------------+
|0  |Bill Chambers   |0               |[100]          |
|1  |Matei Zaharia   |1               |[500, 250, 100]|
|2  |Michael Armbrust|1               |[250, 100]     |
+---+----------------+----------------+---------------+



In [7]:
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")

In [8]:
graduateProgram.show(5,False)

+---+-------+---------------------+-----------+
|id |degree |department           |school     |
+---+-------+---------------------+-----------+
|0  |Masters|School of Information|UC Berkeley|
|2  |Masters|EECS                 |UC Berkeley|
|1  |Ph.D.  |EECS                 |UC Berkeley|
+---+-------+---------------------+-----------+



In [9]:
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [10]:
sparkStatus.show(5, False)

+---+--------------+
|id |status        |
+---+--------------+
|500|Vice President|
|250|PMC Member    |
|100|Contributor   |
+---+--------------+



In [11]:
# COMMAND ----------
joinExpression = person["graduate_program"] == graduateProgram['id']

In [12]:
# COMMAND ----------
wrongJoinExpression = person["name"] == graduateProgram["school"]

In [13]:
# COMMAND ----------
joinType = "inner"
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [14]:
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression).show(8, False)
#Deafault Join is Inner Join

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [15]:
joinType='outer'
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
|null    |null            |null            |null           |2  |Masters|EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [16]:
joinType='outer'
graduateProgram.join(person.withColumnRenamed("id", "personId"), joinExpression, joinType).show(8, False)

+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|id |degree |department           |school     |personId|name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0       |Bill Chambers   |0               |[100]          |
|1  |Ph.D.  |EECS                 |UC Berkeley|1       |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.  |EECS                 |UC Berkeley|2       |Michael Armbrust|1               |[250, 100]     |
|2  |Masters|EECS                 |UC Berkeley|null    |null            |null            |null           |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+



In [17]:
joinType='left_outer'
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [18]:
joinType='left_outer'
graduateProgram.join(person.withColumnRenamed("id", "personId"), joinExpression, joinType).show(8, False)

+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|id |degree |department           |school     |personId|name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0       |Bill Chambers   |0               |[100]          |
|1  |Ph.D.  |EECS                 |UC Berkeley|1       |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.  |EECS                 |UC Berkeley|2       |Michael Armbrust|1               |[250, 100]     |
|2  |Masters|EECS                 |UC Berkeley|null    |null            |null            |null           |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+



In [19]:
joinType='right_outer'
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
|null    |null            |null            |null           |2  |Masters|EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [20]:
joinType='right_outer'
graduateProgram.join(person.withColumnRenamed("id", "personId"), joinExpression, joinType).show(8, False)

+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|id |degree |department           |school     |personId|name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0       |Bill Chambers   |0               |[100]          |
|1  |Ph.D.  |EECS                 |UC Berkeley|1       |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.  |EECS                 |UC Berkeley|2       |Michael Armbrust|1               |[250, 100]     |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+



In [21]:
# COMMAND ----------
gradProgram2 = graduateProgram.union(
    spark.createDataFrame([(0, "Masters", "Duplicated Row", "Duplicated School")]))

In [22]:
gradProgram2.show(5, False)

+---+-------+---------------------+-----------------+
|id |degree |department           |school           |
+---+-------+---------------------+-----------------+
|0  |Masters|School of Information|UC Berkeley      |
|2  |Masters|EECS                 |UC Berkeley      |
|1  |Ph.D.  |EECS                 |UC Berkeley      |
|0  |Masters|Duplicated Row       |Duplicated School|
+---+-------+---------------------+-----------------+



In [23]:
# COMMAND ----------
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId")\
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show(10, False)

+--------+----------------+----------------+---------------+---+--------------+
|personId|name            |graduate_program|spark_status   |id |status        |
+--------+----------------+----------------+---------------+---+--------------+
|0       |Bill Chambers   |0               |[100]          |100|Contributor   |
|1       |Matei Zaharia   |1               |[500, 250, 100]|500|Vice President|
|1       |Matei Zaharia   |1               |[500, 250, 100]|250|PMC Member    |
|1       |Matei Zaharia   |1               |[500, 250, 100]|100|Contributor   |
|2       |Michael Armbrust|1               |[250, 100]     |250|PMC Member    |
|2       |Michael Armbrust|1               |[250, 100]     |100|Contributor   |
+--------+----------------+----------------+---------------+---+--------------+



In [24]:
gradProgDupe = graduateProgram.withColumnRenamed("id","graduate_program")

In [25]:
joinExpression = gradProgDupe["graduate_program"] == person["graduate_program"]

In [26]:
person.join(gradProgDupe, "graduate_program").show()

+----------------+---+----------------+---------------+-------+--------------------+-----------+
|graduate_program| id|            name|   spark_status| degree|          department|     school|
+----------------+---+----------------+---------------+-------+--------------------+-----------+
|               0|  0|   Bill Chambers|          [100]|Masters|School of Informa...|UC Berkeley|
|               1|  1|   Matei Zaharia|[500, 250, 100]|  Ph.D.|                EECS|UC Berkeley|
|               1|  2|Michael Armbrust|     [250, 100]|  Ph.D.|                EECS|UC Berkeley|
+----------------+---+----------------+---------------+-------+--------------------+-----------+



In [27]:
res = person.join(gradProgDupe, joinExpression)

In [28]:
res.show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+



In [29]:
# # res.select("graduate_program").show()
# #Py4JJavaError: An error occurred while calling o138.select.
# : org.apache.spark.sql.AnalysisException: 
#         Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.;

In [30]:
from pyspark.sql.functions import col
res1 = person.join(gradProgDupe, joinExpression).drop(gradProgDupe["graduate_program"])

In [31]:
res2 = person.join(gradProgDupe, joinExpression).drop(person["graduate_program"])

In [32]:
res1.select("graduate_program").show()
res2.select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+

